# Libraries installation on the current python environment

# Pre-processing original data

In [74]:
import csv
def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break

In [75]:
sampleAnno = -5

## Reading from dataset

In [76]:
import os
import pandas as pd
ANNOTATIONS_PATH = 'data'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#with open(ANNOTATIONS_PATH+'/annotations.txt','r') as file:
#    annotations = file.read().splitlines()
#annotations_file = ANNOTATIONS_PATH+'/annotationsVExamples.txt' #'/annotationsVSingle.txt'
#annotations_file = ANNOTATIONS_PATH+'/annotationsVExamples.txt' #'/annotationsVSingle.txt'
annotations_file = ANNOTATIONS_PATH+'/annotationsVSaliency.txt'
#annotations_file = ANNOTATIONS_PATH+'/annotationsVSingle.txt'
with open(annotations_file,'r') as file:
    annotations = file.read().splitlines()[1:]#[line for line in file.read().splitlines() if 'no val' in line]
    
if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
    folder, trial, fragId, side, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
else:
    folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]
sampleData = os.path.join(RAW_PATH,folder,trial+'.tsv')
sampleData = os.path.join(ANNOTATIONS_PATH,folder,trial+'.tsv')
print(sampleData,startSec,endSec)
with open(sampleData,'r') as file:
    numMarkers, columnNames, readerBuffer = custom_header_reader(file)
    data = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))
if data['Time'].iloc[0] > float(startSec) or data['Time'].iloc[-1] < float(endSec):
    raise Exception(f'\x1b[31mNOT FULL FRAMES!!! data starts at: {data["Time"].iloc[0]} of {float(startSec)} and ends at {data["Time"].iloc[-1]} of {float(endSec)}\x1b[0m')
else:
    print("\x1b[32mSTART AND END SECONDS CHECK CORRECT!\x1b[0m")
data.head()

data/paper/t_051_1.tsv 1.45 5.09
START AND END SECONDS CHECK CORRECT!


,Frame,Time,SMPTE,Federico_HeadFront X,Federico_HeadFront Y,Federico_HeadFront Z,Federico_HeadL X,Federico_HeadL Y,Federico_HeadL Z,Federico_HeadR X,...,Federico_LElbowIn Z,Federico_RElbowIn X,Federico_RElbowIn Y,Federico_RElbowIn Z,Federico_LElbowOut X,Federico_LElbowOut Y,Federico_LElbowOut Z,Federico_RElbowOut X,Federico_RElbowOut Y,Federico_RElbowOut Z
0,146,1.45,00:58:07:19,2085.053,885.074,1664.741,2202.249,1005.959,1605.002,2012.395,...,1243.564,1685.494,1053.834,1256.296,2567.492,1126.140,1356.453,1641.730,1087.231,1372.333
1,147,1.46,00:58:07:19,2081.645,882.795,1662.650,2201.749,1002.239,1605.551,2012.432,...,1244.014,1686.325,1056.003,1256.431,2568.492,1124.760,1357.013,1642.773,1089.206,1372.507
2,148,1.47,00:58:07:20,2078.089,880.502,1660.573,2201.145,998.205,1606.019,2012.527,...,1244.406,1687.216,1058.349,1256.533,2569.507,1123.271,1357.468,1643.925,1091.455,1372.606
3,149,1.48,00:58:07:20,2074.250,878.161,1658.258,2200.321,993.857,1606.292,2012.700,...,1244.816,1688.219,1061.008,1256.551,2570.577,1121.650,1357.891,1645.167,1093.981,1372.575
4,150,1.49,00:58:07:20,2070.189,875.806,1655.819,2199.509,989.233,1606.663,2012.846,...,1245.169,1689.296,1063.919,1256.493,2571.803,1120.022,1358.291,1646.489,1096.829,1372.454


## Standardizing column names

In [77]:
print(len(data))
data.iloc[:,3::3].head()

365


,Federico_HeadFront X,Federico_HeadL X,Federico_HeadR X,Federico_Chest X,Federico_WaistLFront X,Federico_WaistRFront X,Federico_LShoulderTop X,Federico_RShoulderTop X,Federico_SpineThoracic2 X,Federico_SpineThoracic12 X,...,Federico_LWristIn X,Federico_RWristIn X,Federico_LWristOut X,Federico_RWristOut X,Federico_LArm X,Federico_RArm X,Federico_LElbowIn X,Federico_RElbowIn X,Federico_LElbowOut X,Federico_RElbowOut X
0,2085.053,2202.249,2012.395,2126.656,2254.420,2013.345,2239.153,2002.797,2116.284,2131.427,...,2797.598,1432.800,2802.582,1415.783,2433.380,1780.928,2546.508,1685.494,2567.492,1641.730
1,2081.645,2201.749,2012.432,2127.121,2255.121,2013.989,2239.954,2003.582,2117.520,2132.960,...,2798.310,1433.587,2803.730,1416.604,2434.387,1782.095,2547.438,1686.325,2568.492,1642.773
2,2078.089,2201.145,2012.527,2127.607,2255.866,2014.698,2240.808,2004.394,2118.806,2134.722,...,2798.970,1434.447,2803.902,1417.532,2435.464,1783.404,2548.468,1687.216,2569.507,1643.925
3,2074.250,2200.321,2012.700,2128.074,2256.645,2015.434,2241.691,2005.210,2120.156,2136.347,...,2799.610,1435.342,2805.182,1418.482,2436.540,1784.812,2549.459,1688.219,2570.577,1645.167
4,2070.189,2199.509,2012.846,2128.552,2257.488,2016.236,2242.651,2006.013,2121.540,2137.974,...,2800.247,1436.406,2806.647,1419.552,2437.740,1786.386,2550.550,1689.296,2571.803,1646.489


In [78]:
[colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]

['Federico_HeadFront',
 'Federico_HeadL',
 'Federico_HeadR',
 'Federico_Chest',
 'Federico_WaistLFront',
 'Federico_WaistRFront',
 'Federico_LShoulderTop',
 'Federico_RShoulderTop',
 'Federico_SpineThoracic2',
 'Federico_SpineThoracic12',
 'Federico_WaistBack',
 'Federico_WaistL',
 'Federico_WaistR',
 'Federico_LThighFrontLow',
 'Federico_RThighFrontLow',
 'Federico_LKneeOut',
 'Federico_LKneeIn',
 'Federico_RKneeOut',
 'Federico_RKneeIn',
 'Federico_LShinFrontHigh',
 'Federico_RShinFrontHigh',
 'Federico_LHeelBack',
 'Federico_RHeelBack',
 'Federico_LAnkleOut',
 'Federico_RAnkleOut',
 'Federico_LForefoot2',
 'Federico_RForefoot2',
 'Federico_LForefoot5',
 'Federico_RForefoot5',
 'Federico_LHand2',
 'Federico_RHand2',
 'Federico_LWristIn',
 'Federico_RWristIn',
 'Federico_LWristOut',
 'Federico_RWristOut',
 'Federico_LArm',
 'Federico_RArm',
 'Federico_LElbowIn',
 'Federico_RElbowIn',
 'Federico_LElbowOut',
 'Federico_RElbowOut']

In [79]:
data.columns = [item.replace('Federico_', '') for item in data.columns]

In [80]:
import pandas as pd
if '_A' in data.columns[3] or '_B' in data.columns[3]:
    data = data[['Frame','Time','SMPTE']+data.filter(like=f'_{side[1]}').columns.tolist()]
    data.columns = [col.replace(f'_{side[1]}','') for col in data.columns]
elif '61A' in data.columns[3] or '61B' in data.columns[3]:
    data = data[['Frame','Time','SMPTE']+data.filter(like=f'61{side[1]}').columns.tolist()]
    data.columns = [col.replace(f'61{side[1]}_','') for col in data.columns]
if '_01' in data.columns[3]:
    data.columns = [column for column in data.columns[:3]]+[column[:-5]+column[-2:] for column in data.columns[3:]]
if 'HeadFront' in data.columns[3] or 'RKneeIn' in data.columns[3]:
    mappings = pd.read_csv("data/raw/Sport42MarkersTo64.csv")
    mappings = mappings.set_index(mappings.columns[0])[mappings.columns[1]].to_dict()
    data.columns = [mappings[col[:-2]]+col[-2:] if col[:-2] in mappings.keys() else col for col in data.columns]
    

In [81]:
list(data.columns)[:3]+list(data.columns)[3::3]

['Frame',
 'Time',
 'SMPTE',
 'ARIEL X',
 'LFHD X',
 'RFHD X',
 'STRN X',
 'LFWT X',
 'RFWT X',
 'LSHO X',
 'RSHO X',
 'C7 X',
 'T10 X',
 'BWT X',
 'LBWT X',
 'RBWT X',
 'LThighFrontLow X',
 'RThighFrontLow X',
 'LKNE X',
 'LKNI X',
 'RKNE X',
 'RKNI X',
 'LShinFrontHigh X',
 'RShinFrontHigh X',
 'LHEL X',
 'RHEL X',
 'LANK X',
 'RANK X',
 'LTOE X',
 'RTOE X',
 'LMT5 X',
 'RMT5 X',
 'LPLM X',
 'RPLM X',
 'LIWR X',
 'RIWR X',
 'LOWR X',
 'ROWR X',
 'LArm X',
 'RArm X',
 'LIEL X',
 'RIEL X',
 'LELB X',
 'RELB X']

## Working on NaNs

In [82]:
print("Total NaNs")
nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(nans) == 0 else '\x1b[31m'
print(color+str(nans)+'\x1b[0m '+ ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Total NaNs
Series([], dtype: float64) 
No Nans


In [83]:
from itertools import groupby
max_nans_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.isna().tolist())))
print("Max contiguous NaNs")
max_nans_per_column = max_nans_per_column[max_nans_per_column > 0].sort_values(ascending=False)
color = '\x1b[32m' if len(max_nans_per_column) == 0 else '\x1b[31m'
print(color+str(max_nans_per_column)+'\x1b[0m ' + ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Max contiguous NaNs
Series([], dtype: int64) 
No Nans


## Working on Zeros

In [84]:
print('Num Zeros')
zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(zeros) == 0 else '\x1b[31m'
print(color+str(zeros)+'\x1b[0m ' + ('\nNo Zeros' if len(zeros)==0 else '\nZeros found!'))
#list(zeros.index)

Num Zeros
RKNE X    3
RKNE Y    3
RKNE Z    3
dtype: int64 
Zeros found!


In [85]:
from scipy.interpolate import interp1d
import numpy as np

for i,column in enumerate(zeros.index.to_list()):
    data[column] = data[column].apply(lambda x: np.nan if x == 0.0 else x)
    
    fmt2 = data[['Time',column]].dropna()
    f = interp1d(fmt2['Time'],fmt2[column],kind='cubic')
    
    # Replace zero values with interpolated values
    data[column] = data['Time'].apply(f)

In [86]:
import numpy as np
#for name_col,num_zeros in zeros.items():
#    if num_zeros < 20:
#        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

# THIS IS A FIX FOR MISSING QTM AT DATA 58-59
for name_col,num_zeros in zeros.items():
    if num_zeros < 20 and name_col != 'Time' and False:
        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

In [87]:
print('Num Zeros')
zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(zeros) == 0 else '\x1b[31m'
print(color+str(zeros)+'\x1b[0m ' + ('\nNo Zeros' if len(zeros)==0 else '\nZeros found!'))
#list(zeros.index)

Num Zeros
Series([], dtype: float64) 
No Zeros


In [88]:
from itertools import groupby
max_zeros_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.eq(0.0).tolist())))
print("Max contiguous Zeros")
max_zeros_per_column = max_zeros_per_column[max_zeros_per_column > 0].sort_values(ascending=False)
color = '\x1b[32m' if len(max_zeros_per_column) == 0 else '\x1b[31m'
print(color+str(max_zeros_per_column)+'\x1b[0m ' + ('\nNo Zeros' if len(max_zeros_per_column)==0 else '\nZeros found!'))

Max contiguous Zeros
Series([], dtype: int64) 
No Zeros


In [89]:
print("Total NaNs")
nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(nans) == 0 else '\x1b[31m'
print(color+str(nans)+'\x1b[0m '+ ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Total NaNs
Series([], dtype: float64) 
No Nans


## Converting to reduced marker set

![fullbodymarkers](../resources/fullmarkerset.png) ![markerstable](../resources/markersetable.png)

nono smarmella e apri tutto

In [90]:
def map_reduce_num_markers(reducedMarkerNames:list):
    markersMap = {reducedMarkerNames[0]:   ['RHEL','RMT1','RMT5','RTOE'],
                  reducedMarkerNames[1]:   ['LHEL','LMT1','LMT5','LTOE'],
                  reducedMarkerNames[2]:   ['RANK'],
                  reducedMarkerNames[3]:   ['LANK'],
                  reducedMarkerNames[4]:   ['RKNE','RKNI'],
                  reducedMarkerNames[5]:   ['LKNE','LKNI'],
                  reducedMarkerNames[6]:   ['RFWT','RBWT'],
                  reducedMarkerNames[7]:   ['RFWT','LFWT','RBWT','LBWT'],
                  reducedMarkerNames[8]:   ['LFWT','LBWT'],
                  reducedMarkerNames[9]:   ['C6','T5','T10','BWT','STRN','CLAV','FWT'],
                  reducedMarkerNames[10]:  ['RPLM','RTHMB','RINDX','RMID','RPNKY'],
                  reducedMarkerNames[11]:  ['LPLM','LTHMB','LINDX','LMID','LPNKY'],
                  reducedMarkerNames[12]:  ['ROWR','RIWR'],
                  reducedMarkerNames[13]:  ['LOWR','LIWR'],
                  reducedMarkerNames[14]:  ['RELB','RIEL','RFRM'],
                  reducedMarkerNames[15]:  ['LELB','LIEL','LFRM'],
                  reducedMarkerNames[16]:  ['RSHO'],
                  reducedMarkerNames[17]:  ['RSHO','LSHO'],
                  reducedMarkerNames[18]:  ['LSHO'],
                  reducedMarkerNames[19]:  ['ARIEL','RFHD','LFHD','RBHD','LBHD']
                  }
    #   removing nan joints
    for nan_joint_name,_ in max_nans_per_column[::3].items():
        nan_joint_name = nan_joint_name[:-2]
        for markerFullList in markersMap.values():
            if nan_joint_name in markerFullList:
                markerFullList.remove(nan_joint_name)
                break
    # removing many contiguous zeros joints
    for zero_joint_name,contiguous_zeros in max_zeros_per_column[::3].items():
        if contiguous_zeros > 20:
            zero_joint_name = zero_joint_name[:-2]
            for markerFullList in markersMap.values():
                if zero_joint_name in markerFullList:
                    markerFullList.remove(zero_joint_name)
                    break
    originalDataColumns = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
    
    #   removing markers not present in the data markers
    for markerFullList in markersMap.values():
        for markerName in list(reversed(markerFullList)):
            if not markerName in originalDataColumns:
                markerFullList.remove(markerName)

    #   removing asymmetries across body
    for right_side_marker_indx,left_side_marker_indx in [(0,1),(2,3),(4,5),(6,8),(10,11),(12,13),(14,15)]: # indices of every marker on the left side and his symmetric
        markersLeft = markersMap[reducedMarkerNames[left_side_marker_indx]]
        markersRight = markersMap[reducedMarkerNames[right_side_marker_indx]]
        for elem in markersLeft:
            if not 'R'+elem[1:] in markersRight:
                markersLeft.remove(elem)
                if right_side_marker_indx == 6: 
                    markersMap[reducedMarkerNames[7]].remove(elem)
        for elem in markersRight:
            if not 'L'+elem[1:] in markersLeft:
                markersRight.remove(elem)
                if left_side_marker_indx == 8: 
                    markersMap[reducedMarkerNames[7]].remove(elem)

    for i,markerFullList in enumerate(markersMap.values()):
        if len(markerFullList) == 0:
            print(markersMap)
            raise Exception(f"can't reconstruct '{reducedMarkerNames[i]}' of 20 markers, because every marker of the full set is missing!!")
    return markersMap

In [91]:
reducedMarkerNames = ['left_foot',      # 0
                      'right_foot',     # 1          
                      'left_ank',       # 2      
                      'right_ank',      # 3                          
                      'left_knee',      # 4                          
                      'right_knee',     # 5                              
                      'left_hip',       # 6                          
                      'hip_central',    # 7                              
                      'right_hip',      # 8                          
                      'spine',          # 9                       
                      'left_hand',      # 10                           
                      'right_hand',     # 11                               
                      'left_wrist',     # 12                               
                      'right_wrist',    # 13                               
                      'left_elbow',     # 14                               
                      'right_elbow',    # 15                               
                      'left_shoulder',  # 16                               
                      'shoulder_center',# 17                                   
                      'right_shoulder', # 18                                   
                      'head']           # 19                       
fullMarkerNames = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
reduced20MarkersToFullMarkers = map_reduce_num_markers(reducedMarkerNames)
reduced20MarkersToFullMarkersX = {key+'_X': [elem+' X' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
reduced20MarkersToFullMarkersY = {key+'_Y': [elem+' Y' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
reduced20MarkersToFullMarkersZ = {key+'_Z': [elem+' Z' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}

posTable = data.iloc[:,3:]

posTableX = posTable.iloc[:,::3]
posTableY = posTable.iloc[:,1::3]
posTableZ = posTable.iloc[:,2::3]

reducedPosTableX = pd.concat([posTableX[reduced20MarkersToFullMarkersX[colName+'_X']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_X' for name in reducedMarkerNames])
reducedPosTableY = pd.concat([posTableY[reduced20MarkersToFullMarkersY[colName+'_Y']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Y' for name in reducedMarkerNames])
reducedPosTableZ = pd.concat([posTableZ[reduced20MarkersToFullMarkersZ[colName+'_Z']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Z' for name in reducedMarkerNames])

reducedPosTableX = (reducedPosTableX - reducedPosTableX.min(axis=None))/(reducedPosTableX.max(axis=None)-reducedPosTableX.min(axis=None))
reducedPosTableY = (reducedPosTableY - reducedPosTableY.min(axis=None))/(reducedPosTableY.max(axis=None)-reducedPosTableY.min(axis=None))
reducedPosTableZ = (reducedPosTableZ - reducedPosTableZ.min(axis=None))/(reducedPosTableZ.max(axis=None)-reducedPosTableZ.min(axis=None))

def xyz_tables_to_xyz_columns(tablesList):
    xTable,yTable,zTable = tablesList
    mergedTable = pd.DataFrame()
    for joint in range(xTable.shape[1]):
        mergedTable = pd.concat([mergedTable,xTable.iloc[:,joint],yTable.iloc[:,joint],zTable.iloc[:,joint]],axis=1)
    return mergedTable

reducedPosTable = xyz_tables_to_xyz_columns([reducedPosTableX,reducedPosTableY,reducedPosTableZ])
reducedPosTable = (reducedPosTable - reducedPosTable.min(axis=None))/(reducedPosTable.max(axis=None)-reducedPosTable.min(axis=None))
reducedPosTable.head()

,left_foot_X,left_foot_Y,left_foot_Z,right_foot_X,right_foot_Y,right_foot_Z,left_ank_X,left_ank_Y,left_ank_Z,right_ank_X,...,left_shoulder_Z,shoulder_center_X,shoulder_center_Y,shoulder_center_Z,right_shoulder_X,right_shoulder_Y,right_shoulder_Z,head_X,head_Y,head_Z
0,0.406741,0.537368,0.000063,0.596469,0.534027,0.001328,0.387071,0.662498,0.015014,0.612249,...,0.874278,0.498080,0.494883,0.878488,0.575083,0.497275,0.882698,0.484347,0.412886,0.977678
1,0.406750,0.537345,0.000098,0.596478,0.534128,0.001265,0.387077,0.662343,0.015164,0.612214,...,0.874296,0.498596,0.494352,0.878605,0.575605,0.496069,0.882914,0.483506,0.410505,0.977266
2,0.406750,0.537305,0.000150,0.596465,0.534140,0.001246,0.387080,0.662223,0.015294,0.612182,...,0.874307,0.499139,0.493753,0.878696,0.576161,0.494764,0.883085,0.482623,0.407974,0.976841
3,0.406769,0.537265,0.000216,0.596483,0.534197,0.001227,0.387096,0.662029,0.015441,0.612152,...,0.874299,0.499693,0.493030,0.878763,0.576737,0.493329,0.883227,0.481648,0.405303,0.976291
4,0.406778,0.537183,0.000268,0.596597,0.534224,0.001202,0.387097,0.661832,0.015548,0.612138,...,0.874246,0.500267,0.492205,0.878794,0.577362,0.491735,0.883343,0.480621,0.402457,0.975695


## Writing to files final clean

In [92]:
import os

PROCESSED_PATH = 'data/reprocessed/'

dest_dir = os.path.join(PROCESSED_PATH,folder)
if not os.path.exists(dest_dir): 
    os.mkdir(dest_dir)
dest_file_path = os.path.join(dest_dir,trial+'_frag'+fragId+'.csv')
if not os.path.isfile(dest_file_path) or True:
    reducedPosTable.to_csv(dest_file_path,index=False)

data/reprocessed/paper


## Plotting reduced markers

In [51]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

posTableY = reducedPosTable.iloc[:,::3]
posTableX = reducedPosTable.iloc[:,1::3]
posTableZ = reducedPosTable.iloc[:,2::3]

#if sampleAnno < 45:
#    posTableY = reducedPosTable.iloc[:,::3]
#    posTableZ = reducedPosTable.iloc[:,1::3]
#    posTableX = reducedPosTable.iloc[:,2::3]
#else:
#    posTableX = reducedPosTable.iloc[:,::3]
#    posTableY = reducedPosTable.iloc[:,1::3]
#    posTableZ = reducedPosTable.iloc[:,2::3]

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
jointsFrom = list(map(lambda x: x-1,jointsFrom))
jointsTo = list(map(lambda x: x-1,jointsTo))
edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
adjacencyMatrix = np.zeros((20,20),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True

plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ)

    # Add edges based on the weight matrix
    for i in range(20):
        for j in range(i + 1, 20):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='gray', linestyle='-', linewidth=1)

    ax.set_xlim([-0.1,0.8])#minMax[0,0],minMax[1,0]])
    ax.set_ylim([-0.1,0.8])#minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('Y', fontsize=12)
    ax.set_zlabel('Z', fontsize=12)
    ax.set_title("Movement "+str(sampleAnno))

    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()

Using matplotlib backend: TkAgg


## Plotting full markers

In [ ]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
if sampleAnno < 45:
    posTableY = posTable.iloc[:,::3]
    posTableZ = posTable.iloc[:,1::3]
    posTableX = posTable.iloc[:,2::3]
else:
    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,1::3]
    posTableZ = posTable.iloc[:,2::3]
    

plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0
column_names = [ col[:-2] for col in list(posTable.columns)[::3] ]

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ)

    for i, name in enumerate(column_names):
        ax.text(filteredX[i], filteredY[i], filteredZ[i], name, fontsize=8, ha='left')


    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('Y', fontsize=12)
    ax.set_zlabel('Z', fontsize=12)
    #ax.set_title("Movement "+str(picked))

    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and pos

In [19]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

minMax[0,:] = [np.nanmin(posTableX.iloc[:1,:].values),np.nanmin(posTableY.iloc[:1,:].values),np.nanmin(posTableZ.iloc[:1,:].values)]
minMax[1,:] = [np.nanmax(posTableX.iloc[:1,:].values),np.nanmax(posTableY.iloc[:1,:].values),np.nanmax(posTableZ.iloc[:1,:].values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=0)
    
    for i, col_name in enumerate(posTableX.columns):
        ax.text(filteredX[i], filteredY[i], filteredZ[i], col_name[:-2], horizontalalignment='center',fontsize=6, color='black')

    ax.set_xticks([minMax[0,0],minMax[1,0]])
    ax.set_yticks([minMax[0,1],minMax[1,1]])
    ax.set_zticks([minMax[0,2],minMax[1,2]])
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])
    
    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


In [242]:
plt.close('all')

# All in One

In [195]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def visualize_a_sample(posTable, sample):
    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,2::3]
    posTableZ = posTable.iloc[:,1::3]
    jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
    jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
    jointsFrom = list(map(lambda x: x-1,jointsFrom))
    jointsTo = list(map(lambda x: x-1,jointsTo))
    edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
    adjacencyMatrix = np.zeros((20,20),dtype=bool)
    adjacencyMatrix[edges[:,0],edges[:,1]] = True

    plt.close('all')
    fig = plt.figure(figsize=(10,8))
    ax:plt.Axes = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=0, azim=0)
    fig.canvas.manager.full_screen_toggle()

    
    minMax = np.zeros((2,3))
    minMax[0,:] = [posTableX.values.min(),posTableY.values.min(),posTableZ.values.min()]
    minMax[1,:] = [posTableX.values.max(),posTableY.values.max(),posTableZ.values.max()]

    # Set appropriate axis limits
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([0,1])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    # Set the window title
    fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

    # Set the initial time index
    time_index = 0
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'cyan']

    # Function to update the plot based on the slider value
    def update_plot(val):
        ax.cla()  # Clear the previous plot

        # Filter the data based on the current time index
        filteredX = posTableX.iloc[val]
        filteredY = posTableY.iloc[val]
        filteredZ = posTableZ.iloc[val]

        ax.scatter(filteredX,filteredY,filteredZ)

        # Add edges based on the weight matrix
        for i in range(20):
            for j in range(i + 1, 20):
                if adjacencyMatrix[i,j]:
                    ax.plot([filteredX[i], filteredX[j]],
                            [filteredY[i], filteredY[j]],
                            [filteredZ[i], filteredZ[j]],
                            color='gray', linestyle='-', linewidth=1)


        ax.set_xlim([minMax[0,0],minMax[1,0]])
        ax.set_ylim([0,1])
        ax.set_zlim([minMax[0,2],minMax[1,2]])

        ax.set_xlabel('X', fontsize=12)
        ax.set_ylabel('Y', fontsize=12)
        ax.set_zlabel('Z', fontsize=12)
        ax.set_title("Movement "+str(sample))

        fig.canvas.draw_idle()

    # Create a slider widget
    slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
    maxValue = posTableX.shape[0]-1
    slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


    # Define a function to update the slider value with the mouse wheel
    def on_scroll(event):
        if event.button == 'down':
            if slider.val + slider.valstep <= maxValue:
                slider.set_val(slider.val + slider.valstep)
        elif event.button == 'up':
            if slider.val - slider.valstep >= 0:
                slider.set_val(slider.val - slider.valstep)


    # Connect the mouse wheel event to the function
    fig.canvas.mpl_connect('scroll_event', on_scroll)


    # Register the update_plot function with the slider widget
    slider.on_changed(update_plot)

    # Initial plot
    update_plot(time_index)

    # Show the plot
    plt.show(block=True)

# Function to find the maximum consecutive zeros values from a mask series
def max_consecutive_zeros(series):
        consecutive_true_count = 0
        max_consecutive_true_count = 0

        for value in series:
            if value:
                consecutive_true_count += 1
                if consecutive_true_count > max_consecutive_true_count:
                    max_consecutive_true_count = consecutive_true_count
            else:
                consecutive_true_count = 0

        return max_consecutive_true_count


Using matplotlib backend: TkAgg


In [206]:
import csv
import os
import pandas as pd
from tqdm.notebook import tqdm
from itertools import groupby

def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break


ANNOTATIONS_PATH = 'data'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#with open(ANNOTATIONS_PATH+'/annotations.txt','r') as file:
#    annotations = file.read().splitlines()
with open(ANNOTATIONS_PATH+'/annotationsVSingle.txt','r') as file:
    annotations = file.read().splitlines()[1:]#[line for line in file.read().splitlines() if 'no val' in line]
    #print(len(annotations))
    
for sampleAnno in tqdm(range(len(annotations)),desc="Compressing markers and saving to csv reprocessed"):
    if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
        folder, trial, fragId, side, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
    else:
        folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]
    sampleData = os.path.join(RAW_PATH,folder,trial+'.tsv')
    with open(sampleData,'r') as file:
        numMarkers, columnNames, readerBuffer = custom_header_reader(file)
        data = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))

    if data['Time'].iloc[0] > float(startSec) or data['Time'].iloc[-1] < float(endSec):
        raise Exception(f'\x1b[31mNOT FULL FRAMES!!! data starts at: {data["Time"].iloc[0]} of {float(startSec)} and ends at {data["Time"].iloc[-1]} of {float(endSec)}\x1b[0m')
    else:
        pass


    if '_A' in data.columns[3] or '_B' in data.columns[3]:
        data = data[['Frame','Time','SMPTE']+data.filter(like=f'_{side[1]}').columns.tolist()]
        data.columns = [col.replace(f'_{side[1]}','') for col in data.columns]
    elif '61A' in data.columns[3] or '61B' in data.columns[3]:
        data = data[['Frame','Time','SMPTE']+data.filter(like=f'61{side[1]}').columns.tolist()]
        data.columns = [col.replace(f'61{side[1]}_','') for col in data.columns]
    if '_01' in data.columns[3]:
        data.columns = [column for column in data.columns[:3]]+[column[:-5]+column[-2:] for column in data.columns[3:]]
    if 'HeadFront' in data.columns[3] or 'RKneeIn' in data.columns[3]:
        mappings = pd.read_csv("data/raw/Sport42MarkersTo64.csv")
        mappings = mappings.set_index(mappings.columns[0])[mappings.columns[1]].to_dict()
        data.columns = [mappings[col[:-2]]+col[-2:] if col[:-2] in mappings.keys() else col for col in data.columns]

    nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)



    max_nans_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.isna().tolist())))

    max_nans_per_column = max_nans_per_column[max_nans_per_column > 0].sort_values(ascending=False)


    zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)

    # THIS IS A FIX FOR MISSING QTM AT DATA 58-59
    if len(zeros.index) > 15:
        for name_col,num_zeros in zeros.items():
            if num_zeros < 50:
                data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)
    if sampleAnno == 1:
        for name_col,num_zeros in zeros.items():
            if num_zeros < 10:
                data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)
    #for name_col,num_zeros in zeros.items():
    #    if num_zeros < 20:
    #        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

    zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)

    max_zeros_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.eq(0.0).tolist())))
    max_zeros_per_column = max_zeros_per_column[max_zeros_per_column > 0].sort_values(ascending=False)

    def map_reduce_num_markers(reducedMarkerNames:list):
        markersMap = {reducedMarkerNames[0]:   ['RHEL','RMT1','RMT5','RTOE'],
                      reducedMarkerNames[1]:   ['LHEL','LMT1','LMT5','LTOE'],
                      reducedMarkerNames[2]:   ['RANK'],
                      reducedMarkerNames[3]:   ['LANK'],
                      reducedMarkerNames[4]:   ['RKNE','RKNI'],#,'RSHN','RShinFrontHigh'],
                      reducedMarkerNames[5]:   ['LKNE','LKNI'],#,'LSHN','LShinFrontHigh'],
                      reducedMarkerNames[6]:   ['RFWT','RBWT'],
                      reducedMarkerNames[7]:   ['RFWT','LFWT','RBWT','LBWT'],
                      reducedMarkerNames[8]:   ['LFWT','LBWT'],
                      reducedMarkerNames[9]:   ['C6','C7','T5','T10','BWT','STRN','CLAV','FWT'],
                      reducedMarkerNames[10]:  ['RPLM','RTHMB','RINDX','RMID','RPNKY'],
                      reducedMarkerNames[11]:  ['LPLM','LTHMB','LINDX','LMID','LPNKY'],
                      reducedMarkerNames[12]:  ['ROWR','RIWR'],
                      reducedMarkerNames[13]:  ['LOWR','LIWR'],
                      reducedMarkerNames[14]:  ['RELB','RIEL','RFRM'],
                      reducedMarkerNames[15]:  ['LELB','LIEL','LFRM'],
                      reducedMarkerNames[16]:  ['RSHO'],
                      reducedMarkerNames[17]:  ['RSHO','LSHO'],#'C7'],
                      reducedMarkerNames[18]:  ['LSHO'],
                      reducedMarkerNames[19]:  ['ARIEL','RFHD','LFHD','RBHD','LBHD']
                      }
        
        # removing nan joints
        for nan_joint_name,_ in max_nans_per_column[::3].items():
            nan_joint_name = nan_joint_name[:-2]
            for markerFullList in markersMap.values():
                if nan_joint_name in markerFullList:
                    markerFullList.remove(nan_joint_name)
                    break

        # removing many contiguous zeros joints
        for zero_joint_name,contiguous_zeros in max_zeros_per_column[::3].items():
            if contiguous_zeros > 20:
                zero_joint_name = zero_joint_name[:-2]
                for markerFullList in markersMap.values():
                    if zero_joint_name in markerFullList:
                        markerFullList.remove(zero_joint_name)
                        break
        originalDataColumns = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]

        # removing markers not present in the data markers
        for markerFullList in markersMap.values():
            for markerName in list(reversed(markerFullList)):
                if not markerName in originalDataColumns:
                    markerFullList.remove(markerName)

        #   removing asymmetries across body
        for right_side_marker_indx,left_side_marker_indx in [(0,1),(2,3),(4,5),(6,8),(10,11),(12,13),(14,15)]: # indices of every marker on the left side and his symmetric
            markersLeft = markersMap[reducedMarkerNames[left_side_marker_indx]]
            markersRight = markersMap[reducedMarkerNames[right_side_marker_indx]]
            for elem in markersLeft:
                if not 'R'+elem[1:] in markersRight:
                    markersLeft.remove(elem)
                    if right_side_marker_indx == 6: 
                        markersMap[reducedMarkerNames[7]].remove(elem)
            for elem in markersRight:
                if not 'L'+elem[1:] in markersLeft:
                    markersRight.remove(elem)
                    if left_side_marker_indx == 8: 
                        markersMap[reducedMarkerNames[7]].remove(elem)

        for i,markerFullList in enumerate(markersMap.values()):
            if len(markerFullList) == 0:
                print(markersMap)
                raise Exception(f"can't reconstruct '{reducedMarkerNames[i]}' of 20 markers, because every marker of the full set is missing!!")
        return markersMap

    reducedMarkerNames = ['right_foot',      # 1
                          'left_foot',       # 2        
                          'right_ank',       # 3      
                          'left_ank',        # 4                          
                          'right_knee',      # 5                          
                          'left_knee',       # 6                              
                          'right_hip',       # 7                          
                          'hip_central',     # 8                              
                          'left_hip',        # 9                          
                          'spine',           # 10                       
                          'right_hand',      # 11                           
                          'left_hand',       # 12                               
                          'right_wrist',     # 13                               
                          'left_wrist',      # 14                               
                          'right_elbow',     # 15                               
                          'left_elbow',      # 16                               
                          'right_shoulder',  # 17                               
                          'shoulder_center', # 18                                   
                          'left_shoulder',   # 19                                   
                          'head']            # 20                       
    fullMarkerNames = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
    reduced20MarkersToFullMarkers = map_reduce_num_markers(reducedMarkerNames)
    reduced20MarkersToFullMarkersX = {key+'_X': [elem+' X' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
    reduced20MarkersToFullMarkersY = {key+'_Y': [elem+' Y' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
    reduced20MarkersToFullMarkersZ = {key+'_Z': [elem+' Z' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}

    posTable = data.iloc[:,3:]

    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,1::3]
    posTableZ = posTable.iloc[:,2::3]

    reducedPosTableX = pd.concat([posTableX[reduced20MarkersToFullMarkersX[colName+'_X']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_X' for name in reducedMarkerNames])
    reducedPosTableY = pd.concat([posTableY[reduced20MarkersToFullMarkersY[colName+'_Y']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Y' for name in reducedMarkerNames])
    reducedPosTableZ = pd.concat([posTableZ[reduced20MarkersToFullMarkersZ[colName+'_Z']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Z' for name in reducedMarkerNames])

    def xyz_tables_to_xyz_columns(tablesList):
        xTable,yTable,zTable = tablesList
        mergedTable = pd.DataFrame()
        for joint in range(xTable.shape[1]):
            mergedTable = pd.concat([mergedTable,xTable.iloc[:,joint],yTable.iloc[:,joint],zTable.iloc[:,joint]],axis=1)
        return mergedTable

    reducedPosTable = xyz_tables_to_xyz_columns([reducedPosTableX,reducedPosTableY,reducedPosTableZ])
    reducedPosTable = (reducedPosTable - reducedPosTable.min(axis=None))/(reducedPosTable.max(axis=None)-reducedPosTable.min(axis=None))
    #reducedPosTable = reducedPosTable.rolling(window=2).mean().fillna(method='bfill')
    
    if reducedPosTable.eq(0.0).apply(max_consecutive_zeros).max() > 1:
        print(sampleAnno, reducedPosTable.eq(0.0).apply(max_consecutive_zeros).max())
        visualize_a_sample(reducedPosTable,sampleAnno)

    PROCESSED_PATH = 'data/reprocessed/'

    dest_dir = os.path.join(PROCESSED_PATH,folder)
    if not os.path.exists(dest_dir): 
        os.mkdir(dest_dir)
    dest_file_path = os.path.join(dest_dir,trial+'_frag'+fragId+'.csv')
    if not os.path.isfile(dest_file_path) or True:
        reducedPosTable.to_csv(dest_file_path,index=False)

Compressing markers and saving to csv reprocessed:   0%|          | 0/60 [00:00<?, ?it/s]